In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

In [2]:
dtype = torch.FloatTensor

In [3]:
sentences = [
    "i like dog",
    "i love coffee",
    "i hate milk"
]

In [11]:
word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word2idx = {word: idx for idx, word in enumerate(word_list)}
idx2word = {idx: word for idx, word in enumerate(word_list)}
n_class = len(word2idx); n_class

7

In [12]:
# TextRNN parameters
batch_size = len(sentences)
n_step = 2 # number of cells (= number of step)
n_hidden = 5 # number of hidden units in one cell

In [13]:
def make_batch(sentences):
    input_batch = []
    target_batch = []
    
    for sent in sentences:
        word = sent.split()
        input = [word2idx[token] for token in word[:-1]]
        target = word2idx[word[-1]]
        
        input_batch.append(np.eye(n_class)[input]) # lookup, one-hot
        target_batch.append(target)
    return input_batch, target_batch

In [20]:
# to torch.Tensor
input_batch, target_batch = make_batch(sentences)
input_batch = Variable(torch.Tensor(input_batch))
target_batch = Variable(torch.LongTensor(target_batch))

In [21]:
class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN, self).__init__()
        
        self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden)
        self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
        self.b = nn.Parameter(torch.randn([n_class]).type(dtype))
        
    def forward(self, hidden, X):
        X = X.transpose(0, 1) # X: [n_step, batch_size, n_class]
        outputs, hidden = self.rnn(X, hidden)
        # outputs : [n_step, batch_size, num_directions(=1) * n_hidden]
        # hidden : [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        outputs = outputs[-1]
        model = torch.mm(outputs, self.W) + self.b # model: [batch_size, n_class]
        return model

In [22]:
model = TextRNN()

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [24]:
# Training
for epoch in range(5000):
    optimizer.zero_grad()
    
    # hidden : [num_layers * num_directions, batch, hidden_size]
    hidden = Variable(torch.zeros(1, batch_size, n_hidden))
    # input_batch : [batch_size, n_step, n_class]
    output = model(hidden, input_batch)
    
    # output : [batch_size, n_class], target_batch" [batch_size] (LongTensor not one-hot)
    loss = criterion(output, target_batch)
    if (epoch+1) % 500 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

Epoch: 0500 cost = 0.384173
Epoch: 1000 cost = 0.077224
Epoch: 1500 cost = 0.029627
Epoch: 2000 cost = 0.015384
Epoch: 2500 cost = 0.009227
Epoch: 3000 cost = 0.006000
Epoch: 3500 cost = 0.004103
Epoch: 4000 cost = 0.002902
Epoch: 4500 cost = 0.002100
Epoch: 5000 cost = 0.001546


In [25]:
input = [sent.split()[:2] for sent in sentences]

In [ ]:
# Predict
hidden = Variable(torch.zeros(1, batch_size, n_hidden))
pred = model(hidden, input_batch)